In [1]:
##Import
from Bio import SeqIO
import pandas as pd
from collections import Counter
import fuzzysearch
import matplotlib as plt
import itertools
import pickle
import demultiplexing_module
import gzip
plt.rcParams['pdf.fonttype'] = 42

In [2]:
##Load key of conditions and files
samples = pd.read_excel("gp17_expt_file_key.xlsx")[["condition","run","file","site1_L","site1_R","site1_wt","site1_pos","site1_neut","site2_L","site2_R","site2_wt","site2_pos","site2_neut","site3_L","site3_R","site3_wt","site3_pos","site3_neut","site4_L","site4_R","site4_wt","site4_pos","site4_neut"]]

In [ ]:
samples

In [4]:
##Globals
outcomes_dict = {} #editing per site
outcomes_dict_complete = {} #matched outcomes across a site
outcomes_dict_select = {} #no - outcomes
site_outcomes = 'wpn-/' #wild-type, positive, neutral, neither, no site
fewer_outcomes = 'wpn'

In [5]:
##Defs
def extract_and_match(read,index,rep):
    across_sites = []
    for site in range(1,5):
        if site == 1 : #these sites don't have any other near matches and seem to work easily
            left_inside = fuzzysearch.find_near_matches(samples.iloc[index]["site%s_L" % site],read,max_l_dist=1,max_deletions=0,max_insertions=0,max_substitutions=1)
            right_inside = fuzzysearch.find_near_matches(samples.iloc[index]["site%s_R" % site],read,max_l_dist=1,max_deletions=0,max_insertions=0,max_substitutions=1)
        if site == 4: #these sites don't have any other near matches and seem to work easily
            left_inside = fuzzysearch.find_near_matches(samples.iloc[index]["site%s_L" % site],read,max_l_dist=2,max_deletions=0,max_insertions=0,max_substitutions=2)
            right_inside = fuzzysearch.find_near_matches(samples.iloc[index]["site%s_R" % site],read,max_l_dist=2,max_deletions=0,max_insertions=0,max_substitutions=2)
        if site == 2: #left side has a near match, so making it longer and allowing more mismatches
            left_inside = fuzzysearch.find_near_matches(samples.iloc[index]["site%s_L" % site],read,max_l_dist=2,max_deletions=0,max_insertions=0,max_substitutions=2)
            right_inside = fuzzysearch.find_near_matches(samples.iloc[index]["site%s_R" % site],read,max_l_dist=4,max_deletions=0,max_insertions=0,max_substitutions=4) 
        if site == 3:
            left_inside = fuzzysearch.find_near_matches(samples.iloc[index]["site%s_L" % site],read,max_l_dist=3,max_deletions=0,max_insertions=0,max_substitutions=3)
            right_inside = fuzzysearch.find_near_matches(samples.iloc[index]["site%s_R" % site],read,max_l_dist=3,max_deletions=0,max_insertions=0,max_substitutions=3)             
        if len(left_inside) == 1 and len(right_inside) == 1:
            var_nt = read[left_inside[0].end:right_inside[0].start]
            if var_nt == samples.iloc[index]["site%s_wt" % site]:
                across_sites.append('w')
            elif var_nt == samples.iloc[index]["site%s_pos" % site]:
                across_sites.append('p')
            elif var_nt == samples.iloc[index]["site%s_neut" % site]:
                across_sites.append('n')
            else:
                across_sites.append('-')
        else:
            across_sites.append('/')
    return tuple(across_sites)

In [ ]:
#step through samples
for i in samples.index:
    sample_i = int(i)
    outcomes_dict[samples['file'][i]] = {}
    outcomes_dict_complete[samples['file'][i]] = {}
    outcomes_dict[samples['file'][i]] = {}
    outcomes_dict_complete[samples['file'][i]] = {}
    outcomes_dict_select[samples['file'][i]] = {}
    for product in itertools.product(site_outcomes, repeat=4):
         outcomes_dict_complete[samples['file'][i]][product] = 0
    for product in itertools.product(fewer_outcomes, repeat=4):
         outcomes_dict_select[samples['file'][i]][product] = 0
    for site in range(1,5):
        outcomes_dict[samples['file'][i]][site] = {'wt':0,'pos_ed':0,'neut_ed':0,'neither':0,'nosite':0}
    all_reads_str = []
    read_counter = []
#     fastq_reads = "./%s_R1.fastq" % samples['file'][i]
    fastq_reads = demultiplexing_module.get_file_path(samples['run'][i],samples['file'][i])
    print (fastq_reads[1])
    try:
#         for seq_record in SeqIO.parse(fastq_reads, "fastq"):
        with gzip.open(fastq_reads[1], "rt") as handle: 
            for seq_record in SeqIO.parse(handle, "fastq"):
                all_reads_str.append(str(seq_record.seq))
            read_counter = Counter(all_reads_str)
            for read in read_counter:
                outcomes = extract_and_match(read,i,samples['file'])
                outcomes_dict_complete[samples['file'][i]][outcomes] += read_counter[read]
                try: outcomes_dict_select[samples['file'][i]][outcomes] += read_counter[read]
                except KeyError: pass     
                for position,outcome in enumerate(outcomes):
                    if outcome == 'w':
                        outcomes_dict[samples['file'][i]][position+1]['wt'] += read_counter[read]
                    elif outcome == 'p':
                        outcomes_dict[samples['file'][i]][position+1]['pos_ed'] += read_counter[read]
                    elif outcome == 'n':
                        outcomes_dict[samples['file'][i]][position+1]['neut_ed'] += read_counter[read] 
                    elif outcome == '-':
                        outcomes_dict[samples['file'][i]][position+1]['neither'] += read_counter[read]
                    elif outcome == '/':
                        outcomes_dict[samples['file'][i]][position+1]['nosite'] += read_counter[read]
            print(samples['file'][i])
    except IOError: #this happens when a file is missing
        print("%s missing" % samples['file'][i])

In [7]:
##calculate summary data
Ed_Per = {}
No_Site_Per = {}
Total_Count = {}
Neither_Per = {}

Ed_Per['Site1_pos'] = []
Ed_Per['Site2_pos'] = []
Ed_Per['Site3_pos'] = []
Ed_Per['Site4_pos'] = []

Ed_Per['Site1_neut'] = []
Ed_Per['Site2_neut'] = []
Ed_Per['Site3_neut'] = []
Ed_Per['Site4_neut'] = []

Neither_Per['Site1'] = []
Neither_Per['Site2'] = []
Neither_Per['Site3'] = []
Neither_Per['Site4'] = []

No_Site_Per['Site1'] = []
No_Site_Per['Site2'] = []
No_Site_Per['Site3'] = []
No_Site_Per['Site4'] = []

Total_Count['Site1'] = []
Total_Count['Site2'] = []
Total_Count['Site3'] = []
Total_Count['Site4'] = []

for i in samples.index:
    for site in range(1,5):
        try:
            Ed_Per['Site%s_pos' % site].append((float(outcomes_dict[samples['file'][i]][site]['pos_ed']) / (outcomes_dict[samples['file'][i]][site]['pos_ed']+outcomes_dict[samples['file'][i]][site]['neut_ed']+outcomes_dict[samples['file'][i]][site]['wt']))*100)
        except ZeroDivisionError:
            Ed_Per['Site%s_pos' % site].append("div0")
        try:
            Ed_Per['Site%s_neut' % site].append((float(outcomes_dict[samples['file'][i]][site]['neut_ed']) / (outcomes_dict[samples['file'][i]][site]['pos_ed']+outcomes_dict[samples['file'][i]][site]['neut_ed']+outcomes_dict[samples['file'][i]][site]['wt']))*100)
        except ZeroDivisionError:
            Ed_Per['Site%s_neut' % site].append("div0")
        try:
            Neither_Per['Site%s' % site].append((float(outcomes_dict[samples['file'][i]][site]['neither']) / (outcomes_dict[samples['file'][i]][site]['neither']+outcomes_dict[samples['file'][i]][site]['pos_ed']+outcomes_dict[samples['file'][i]][site]['neut_ed']+outcomes_dict[samples['file'][i]][site]['wt']))*100)
        except ZeroDivisionError:
            Neither_Per['Site%s' % site].append("div0")   
        try:
            No_Site_Per['Site%s' % site].append((float(outcomes_dict[samples['file'][i]][site]['nosite']) / (outcomes_dict[samples['file'][i]][site]['nosite']+outcomes_dict[samples['file'][i]][site]['neither']+outcomes_dict[samples['file'][i]][site]['pos_ed']+outcomes_dict[samples['file'][i]][site]['neut_ed']+outcomes_dict[samples['file'][i]][site]['wt']))*100)
        except ZeroDivisionError:
            No_Site_Per['Site%s' % site].append("div0")  
        #total count per site excludes the nosite counts where the flanking sequences were not identified
        Total_Count['Site%s' % site].append(outcomes_dict[samples['file'][i]][site]['neither']+outcomes_dict[samples['file'][i]][site]['pos_ed']+outcomes_dict[samples['file'][i]][site]['neut_ed']+outcomes_dict[samples['file'][i]][site]['wt'])

In [ ]:
for i in samples.index:
    for site in range(1,5):
        print (samples['condition'][i], 'site %s' % site)
        print ('number of missing sites: %s' % outcomes_dict[samples['file'][i]][site]['nosite'])
        print (outcomes_dict[samples['file'][i]][site]['neither'],outcomes_dict[samples['file'][i]][site]['pos_ed'],outcomes_dict[samples['file'][i]][site]['neut_ed'],outcomes_dict[samples['file'][i]][site]['wt'])
        print ('total seqs analyzed: %s' % (outcomes_dict[samples['file'][i]][site]['neither']+outcomes_dict[samples['file'][i]][site]['pos_ed']+outcomes_dict[samples['file'][i]][site]['neut_ed']+outcomes_dict[samples['file'][i]][site]['wt']))
        print ('\n')

In [9]:
samples.insert(2,'Site1_Pos_Edit_Percent',Ed_Per['Site1_pos'])
samples.insert(3,'Site2_Pos_Edit_Percent',Ed_Per['Site2_pos'])
samples.insert(4,'Site3_Pos_Edit_Percent',Ed_Per['Site3_pos'])
samples.insert(5,'Site4_Pos_Edit_Percent',Ed_Per['Site4_pos'])

samples.insert(6,'Site1_Neut_Edit_Percent',Ed_Per['Site1_neut'])
samples.insert(7,'Site2_Neut_Edit_Percent',Ed_Per['Site2_neut'])
samples.insert(8,'Site3_Neut_Edit_Percent',Ed_Per['Site3_neut'])
samples.insert(9,'Site4_Neut_Edit_Percent',Ed_Per['Site4_neut'])

samples.insert(10,'Site1_NeitherNonWT_Percent',Neither_Per['Site1'])
samples.insert(11,'Site2_NeitherNonWT_Percent',Neither_Per['Site2'])
samples.insert(12,'Site3_NeitherNonWT_Percent',Neither_Per['Site3'])
samples.insert(13,'Site4_NeitherNonWT_Percent',Neither_Per['Site4'])

samples.insert(14,'Site1_Missing_Percent',No_Site_Per['Site1'])
samples.insert(15,'Site2_Missing_Percent',No_Site_Per['Site2'])
samples.insert(16,'Site3_Missing_Percent',No_Site_Per['Site3'])
samples.insert(17,'Site4_Missing_Percent',No_Site_Per['Site4'])

samples.insert(18,'Site1_Total_Count',Total_Count['Site1'])
samples.insert(19,'Site2_Total_Count',Total_Count['Site2'])
samples.insert(20,'Site3_Total_Count',Total_Count['Site3'])
samples.insert(21,'Site4_Total_Count',Total_Count['Site4'])

In [ ]:
samples

In [11]:
##output
samples.to_excel("expt_editing_analysis_output.xlsx")

In [12]:
outcomes_dict_all_sites_seq = {} #removes the '-' or '/' containing sequences (where one or more sites isn't cleanly sequenced)

for sample_key in outcomes_dict_complete:
    outcomes_dict_all_sites_seq[sample_key] = {k:v for k,v in outcomes_dict_complete[sample_key].items() if '-' not in k and '/' not in k}

In [13]:
#Turn outcomes dict into percent
outcomes_dict_all_sites_seq_percent = {}

for sample_key in outcomes_dict_all_sites_seq:
    outcomes_dict_all_sites_seq_percent[sample_key] = {}
    for outcome in outcomes_dict_all_sites_seq[sample_key]:
        outcomes_dict_all_sites_seq_percent[sample_key][outcome] = 100*(float(outcomes_dict_all_sites_seq[sample_key][outcome])/sum(outcomes_dict_all_sites_seq[sample_key].values()))

In [14]:
#pickle the dict
pickle.dump(outcomes_dict_all_sites_seq_percent,open("pickle_output.p","wb" ))